In [8]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from torch import nn
from torch import optim
from torch.distributions.categorical import Categorical

from tetris_gymnasium.envs.tetris import Tetris
import gymnasium as gym

sns.set()

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

## Neural Network for Actor (Policy) and Critic (Value) Model

In [ ]:
# Policy and value model
class ActorCriticNetwork(nn.Module):
    def __init__(self, obs_space_size, action_space_size):
        super().__init__()

        self.shared_layers = nn.Sequential(
            nn.Linear(obs_space_size, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        

        self.policy_layers = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, action_space_size)
        )
        

        self.value_layers = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )
    
    def value(self, obs):
        z = self.shared_layers(obs)
        value = self.value_layers(z)
        return value
        
    def policy(self, obs):
        z = self.shared_layers(obs)
        policy_logits = self.policy_layers(z)
        return policy_logits

    def forward(self, obs):
        z = self.shared_layers(obs)
        policy_logits = self.policy_layers(z)
        value = self.value_layers(z)
        return policy_logits, value


## PPO Training

In [28]:
class PPOTrainer():
  def __init__(self,
              actor_critic,
              ppo_clip_val=0.4,
              target_kl_div=0.01,
              max_policy_train_iters=80,
              value_train_iters=80,
              policy_lr=3e-4,
              value_lr=3e-4):
    self.ac = actor_critic
    self.ppo_clip_val = ppo_clip_val
    self.target_kl_div = target_kl_div
    self.max_policy_train_iters = max_policy_train_iters
    self.value_train_iters = value_train_iters

    policy_params = list(self.ac.shared_layers.parameters()) + \
        list(self.ac.policy_layers.parameters())
    self.policy_optim = optim.Adam(policy_params, lr=policy_lr)

    value_params = list(self.ac.shared_layers.parameters()) + \
        list(self.ac.value_layers.parameters())
    self.value_optim = optim.Adam(value_params, lr=value_lr)

  def train_policy(self, obs, acts, old_log_probs, gaes):
    for _ in range(self.max_policy_train_iters):
      self.policy_optim.zero_grad()

      new_logits = self.ac.policy(obs)
      new_logits = Categorical(logits=new_logits)
      new_log_probs = new_logits.log_prob(acts)

      policy_ratio = torch.exp(new_log_probs - old_log_probs)
      clipped_ratio = policy_ratio.clamp(
          1 - self.ppo_clip_val, 1 + self.ppo_clip_val)
      
      clipped_loss = clipped_ratio * gaes
      full_loss = policy_ratio * gaes
      policy_loss = -torch.min(full_loss, clipped_loss).mean()

      policy_loss.backward()
      self.policy_optim.step()

      kl_div = (old_log_probs - new_log_probs).mean()
      if kl_div >= self.target_kl_div:
        break

  def train_value(self, obs, returns):
    for _ in range(self.value_train_iters):
      self.value_optim.zero_grad()

      values = self.ac.value(obs)
      value_loss = (returns - values) ** 2
      value_loss = value_loss.mean()

      value_loss.backward()
      self.value_optim.step()

In [ ]:
def discount_rewards(rewards, gamma=0.99):
    new_rewards = [float(rewards[-1])]
    for i in reversed(range(len(rewards)-1)):
        new_rewards.append(float(rewards[i]) + gamma * new_rewards[-1])
    return np.array(new_rewards[::-1])

def calculate_gaes(rewards, values, gamma=0.99, decay=0.97):
    next_values = np.concatenate([values[1:], [0]])
    deltas = [rew + gamma * next_val - val for rew, val, next_val in zip(rewards, values, next_values)]

    gaes = [deltas[-1]]
    for i in reversed(range(len(deltas)-1)):
        gaes.append(deltas[i] + decay * gamma * gaes[-1])

    return np.array(gaes[::-1])

In [19]:
# Preprocess observation function
def preprocess_observation(observation):
    board = observation['board'].flatten()
    active_tetromino = observation['active_tetromino_mask'].flatten()
    holder = observation['holder'].flatten()
    queue = observation['queue'].flatten()
    processed_observation = np.concatenate((board, active_tetromino, holder, queue))
    processed_observation = processed_observation / 9.0
    return processed_observation

In [ ]:
def rollout(model, env, max_steps=1000):
    ### Create data storage
    train_data = [[], [], [], [], []] # obs, act, reward, values, act_log_probs
    obs, _ = env.reset()

    ep_reward = 0
    for _ in range(max_steps):
        obs = preprocess_observation(obs)
        logits, val = model(torch.tensor([obs], dtype=torch.float32,
                                         device=DEVICE))
        act_distribution = Categorical(logits=logits)
        act = act_distribution.sample()
        act_log_prob = act_distribution.log_prob(act).item()

        act, val = act.item(), val.item()

        next_obs, reward, done, _, _ = env.step(act)

        for i, item in enumerate((obs, act, reward, val, act_log_prob)):
          train_data[i].append(item)

        obs = next_obs
        ep_reward += reward
        if done:
            break

    train_data = [np.asarray(x) for x in train_data]

    ### Do train data filtering
    train_data[3] = calculate_gaes(train_data[2], train_data[3])

    return train_data, ep_reward

In [25]:
env = gym.make("tetris_gymnasium/Tetris")
obs, _ = env.reset()
obs = preprocess_observation(obs)
len(obs)
env.action_space.n
model = ActorCriticNetwork(len(obs), env.action_space.n)
model = model.to(DEVICE)
train_data, reward = rollout(model, env) # Test rollout function

In [32]:
# Define training params
n_episodes = 15000
print_freq = 20

ppo = PPOTrainer(
    model,
    policy_lr = 3e-4,
    value_lr = 1e-3,
    target_kl_div = 0.02,
    max_policy_train_iters = 40,
    value_train_iters = 40)

In [33]:
# Training loop
ep_rewards = []
for episode_idx in range(n_episodes):
  # Perform rollout
  train_data, reward = rollout(model, env)
  ep_rewards.append(reward)

  # Shuffle
  permute_idxs = np.random.permutation(len(train_data[0]))

  # Policy data
  obs = torch.tensor(train_data[0][permute_idxs],
                     dtype=torch.float32, device=DEVICE)
  acts = torch.tensor(train_data[1][permute_idxs],
                      dtype=torch.int32, device=DEVICE)
  gaes = torch.tensor(train_data[3][permute_idxs],
                      dtype=torch.float32, device=DEVICE)
  gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8) # NORMALIZE
  act_log_probs = torch.tensor(train_data[4][permute_idxs],
                               dtype=torch.float32, device=DEVICE)

  # Value data
  returns = discount_rewards(train_data[2])[permute_idxs]
  returns = torch.tensor(returns, dtype=torch.float32, device=DEVICE)

  # Train model
  ppo.train_policy(obs, acts, act_log_probs, gaes)
  ppo.train_value(obs, returns)

  if (episode_idx + 1) % print_freq == 0:
    print('Episode {} | Avg Reward {:.1f}'.format(
        episode_idx + 1, np.mean(ep_rewards[-print_freq:])))

Episode 20 | Avg Reward 12.2
Episode 40 | Avg Reward 10.4
Episode 60 | Avg Reward 10.8
Episode 80 | Avg Reward 11.2
Episode 100 | Avg Reward 11.3
Episode 120 | Avg Reward 10.1
Episode 140 | Avg Reward 9.9
Episode 160 | Avg Reward 9.8
Episode 180 | Avg Reward 10.8
Episode 200 | Avg Reward 10.2
Episode 220 | Avg Reward 13.9
Episode 240 | Avg Reward 12.3
Episode 260 | Avg Reward 12.6
Episode 280 | Avg Reward 12.1
Episode 300 | Avg Reward 10.5
Episode 320 | Avg Reward 12.5
Episode 340 | Avg Reward 10.5
Episode 360 | Avg Reward 13.3
Episode 380 | Avg Reward 13.9
Episode 400 | Avg Reward 12.1
Episode 420 | Avg Reward 12.4
Episode 440 | Avg Reward 13.0
Episode 460 | Avg Reward 12.2
Episode 480 | Avg Reward 11.8
Episode 500 | Avg Reward 13.9
Episode 520 | Avg Reward 11.9
Episode 540 | Avg Reward 10.3
Episode 560 | Avg Reward 11.3
Episode 580 | Avg Reward 12.3
Episode 600 | Avg Reward 12.7
Episode 620 | Avg Reward 12.9
Episode 640 | Avg Reward 10.7
Episode 660 | Avg Reward 10.8
Episode 680 | Av

KeyboardInterrupt: 